In [ ]:
from time import time, sleep
from bigone.client import Client
import datetime
import telegram

api_key = "231a715c-688b-44a9-a154-73821b91e0ec"
api_secret = "475B00A9A41B8AA96580982187489D61372799D12523CABA70C4EAD70787BD8D"

my_token = '740436686:AAHi8tQoTPcoJilPHTCEPTGzZaOYU0IPdeg'
chat_id = '760152261'

client = Client(api_key, api_secret)
bot = telegram.Bot(token = my_token)

message = '트레이딩 봇 Start!'
bot.sendMessage(chat_id=chat_id, text=message)


is_buy = True

# 아래 3줄은 처음에 구매가격이 있다고 가정하기 위한 코드
not_markets = client.get_ticker('BLACK-EOS')
buy_price = float(not_markets['close'])


# 처음에 판매가격이 없으면 코드오류남. 시가가 0보다 작아질순 없으므로 임의로 0으로 설정함.
# 아래코드에서 매수조건이 처음에 판매했던 가격보다 싸져야 구매. 
# 따라서 처음에 0 으로하게두면, 처음에 구매를 안하게되고 우선 판매를 해야 구매가 가능하게됨.
sell_price = 0

# 매도가 먼저 이루어지기 때문에 매도할때의 time이 이미 있다고 가정하기 위한 코드.
sell_time = datetime.datetime.now()

#bigone 거래수수료 (o.1%)
fee = 0.1

#초기 매도 및 매수 revenue 모두 0으로 설정
total_sell_revenue = 0
total_buy_revenue = 0

#프로그램 시작 시간
start_time = datetime.datetime.now()
print("자동매매 시작시간 = ", start_time)

#프로그램 시작 때, 'BLACK-EOS' 시가
start_markets = client.get_ticker('BLACK-EOS')
start_price = float(start_markets['close'])
start_expected_price = start_price * 1.05
print("자동매매 시작시점 BLACK-EOS 종가 = ",round(start_price,4), "& 첫 매도 가격 = ",round(start_expected_price,4),'이상')

start_message = 'BLACK-EOS 자동매매 GO! 시작 시간 : {} \n 시작 종가 : {} \n 첫 매도 가격 : {} 이상'.format(start_time, round(start_price,4), round(start_expected_price,4))
bot.sendMessage(chat_id=chat_id, text=start_message)

while True:
    
    
    markets2 = client.get_ticker('BLACK-EOS')
    price2 = float(markets2['close'])
    
    
    
    # 매도는 구입가격에 5프로 이상일때만 가능하게 만듬.
    if price2 >= (buy_price * 1.05) and is_buy == True:
        
        revenue = ((price2-buy_price)/buy_price)*100 - fee
        print("매도조건임. 판매가격 =", round(price2,4), "구매했던 가격 =", round(buy_price,4), "가격차이 =",round(price2-buy_price,4), "수익률 =",round(revenue,2),"%")
        is_buy = False
        sell_price = price2
        sell_time = datetime.datetime.now()
        print("판매시간 =", sell_time)
        
        total_sell_revenue += revenue
        print("매도시 총 수익률 =", round(total_sell_revenue,2),"%")
        
        full_revenue = total_sell_revenue + total_buy_revenue
        print("총 자동거래 수익률 =", round(full_revenue,2),"%")
        
        sell_message = '매도함. 판매가격 : {} \n 구입했던 가격 : {} \n 매도 수익률 : {}% \n 총 매도 수익률 : {}% \n 총 자동거래 수익률 : {}%'.format(round(price2,4), round(buy_price,4), round(revenue,2), round(total_sell_revenue,2), round(full_revenue,2))
        bot.sendMessage(chat_id=chat_id, text=sell_message)        
    
    # 매수는 판매가격보다 5프로 미만일때만 매수가 가능. (조건: 매도가 처음에 이루어지고 발생)
    elif price2 <= (sell_price*0.95) and is_buy == False:
          
        revenue2 = ((sell_price-price2)/price2)*100 - fee
        print("매수조건임. \n 구입가격 =", round(price2,4), "판매했던 가격 =",round(sell_price,4), "가격차이 =",round(sell_price-price2,4), "수익률 =",round(revenue2,2),"%")
        is_buy = True
        buy_price = price2
        buy_time = datetime.datetime.now()
        print("구매시간 =", buy_time)
            
        total_buy_revenue += revenue2
        print("매수시 총 수익률 =",round(total_buy_revenue,2),"%")
        
        full_revenue = total_sell_revenue + total_buy_revenue
        print("총 자동거래 수익률 =", round(full_revenue,2),"%")
        
        buy_message = '매수함. \n 구입가격 : {} \n 판매했던 가격 : {} \n 매수 수익률 : {}% \n 총 매수 수익률 : {}% \n 총 자동거래 수익률 : {}%'.format(round(price2,4), round(sell_price,4), round(revenue2,2), round(total_buy_revenue,2), round(full_revenue,2))
        bot.sendMessage(chat_id=chat_id, text=buy_message)
    
    # 만약 종가가 구매가격보다 30프로 이하가된다면, 손해보더라도 손절한다. (조건: 첫 매도를 안하고 30프로떨어져도 손절)
    elif price2 <= (buy_price * 0.7) and is_buy == True:
        loss = -(((price2 - buy_price)/buy_price)*100 - fee)
        print("손절조건임. 판매가격 =", round(price2,4), "구매했던 가격 =", round(buy_price,4), "가격차이 =",round(price2-buy_price,4), "손해률 =",round(loss,2),"%")
        loss_time = datetime.datetime.now()
        print("손절시간 =", loss_time)

        full_revenue = total_sell_revenue + total_buy_revenue + loss
        print("총 자동거래 수익률 =", round(full_revenue,2),"%")
        
        loss_message = '손절함. 자동거래 종료. \n 판매가격 : {} \n 구입했던 가격 : {} \n 손절 손해률 : {}% \n 총 자동거래 수익률 : {}%'.format(round(price2,4), round(buy_price,4), round(loss,2), round(full_revenue,2))
        bot.sendMessage(chat_id=chat_id, text=loss_message)
        
        #손절하면 매도하고 자동거래 종료
        break
        
        
        
            
    sleep(60)